In [5]:
fastfood_prices = {
    "Coke": 1.99,
    "Fries": 2.49,
    "Hamburger": 3.99,
    "Cheeseburger": 4.49,
    "Chicken Nuggets": 4.99,
    "Ice Cream Sundae": 2.99,
    "Salad": 3.99,
    "Fishburger": 4.99,
    "Hot Dog": 3.49,
    "Milkshake": 2.99
}
product_names_string = ', '.join(fastfood_prices.keys())

In [6]:
from openai import OpenAI
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

client = OpenAI()

audio_file = open("speech.mp3", "rb")
transcript = client.audio.transcriptions.create(
  model="whisper-1",
  file=audio_file
)

In [7]:
transcript.text

'Hello, I want to buy two cheeseburgers, one package of large fries and two big cokes, thank you.'

In [8]:
def calculate_total_price(**kwargs):
    products_with_quantities = kwargs.get('products_with_quantities', [])
    total_price = sum(fastfood_prices[product['name']] * product['quantity'] for product in products_with_quantities)
    return total_price


In [9]:
products_to_buy = {"products_with_quantities": [{"name": "Coke", "quantity": 1}, {"name": "Hot Dog", "quantity": 1}]}

final_price = calculate_total_price(**products_to_buy)
print(f"Der Gesamtpreis beträgt: {final_price}€")

Der Gesamtpreis beträgt: 5.48€


In [10]:
tools = [{
    "type": "function",
    "function": {
        "name": "calculate_total_price",
        "description": "Calculates the total price for a list of products with specified quantities, based on a predefined price list.",
        "parameters": {
            "type": "object",
            "properties": {
                "products_with_quantities": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "name": {
                                "type": "string",
                                "description": "The name of the product."
                            },
                            "quantity": {
                                "type": "integer",
                                "description": "The quantity of the product."
                            }
                        },
                        "required": ["name", "quantity"]
                    },
                    "description": "A list of products with their names and quantities. Each item is an object with 'name' and 'quantity' properties."
                }
            },
            "required": ["products_with_quantities"]
        }
    }
}]


In [11]:
from openai import OpenAI

GPT_MODEL = "gpt-3.5-turbo-0613"
client = OpenAI()

In [21]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": f"You are an assistant to reformulate textinut of a user. He orders products. You identify the following products {product_names_string} in the request and rename it if there is an error. Example: 'hat dog' -> 'Hot Dog' Also reformulate or create numbers from the request. Example: 'A Hot Dog' -> '1 Hot Dog'. Make sure you don´t make up any sizes, like small, medium or large. Double check the new message only contains the exact naming of the products"},
    {"role": "user", "content": transcript.text}
  ],
)
new_question = completion.choices[0].message
print(new_question)

ChatCompletionMessage(content='Hello, I want to buy 2 Cheeseburgers, 1 Fries and 2 Cokes, thank you.', role='assistant', function_call=None, tool_calls=None)


In [22]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": new_question.content}
  ],
  tools=tools
)

print(completion.choices[0].message)

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_lQY1y34FVakjhhZY6TatPZQs', function=Function(arguments='{"products_with_quantities":[{"name":"Cheeseburger","quantity":2},{"name":"Fries","quantity":1},{"name":"Coke","quantity":2}]}', name='calculate_total_price'), type='function')])


In [23]:
message = completion.choices[0].message
message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_lQY1y34FVakjhhZY6TatPZQs', function=Function(arguments='{"products_with_quantities":[{"name":"Cheeseburger","quantity":2},{"name":"Fries","quantity":1},{"name":"Coke","quantity":2}]}', name='calculate_total_price'), type='function')])

In [24]:
print(message.tool_calls[0].function.name)
print(message.tool_calls[0].function.arguments)

calculate_total_price
{"products_with_quantities":[{"name":"Cheeseburger","quantity":2},{"name":"Fries","quantity":1},{"name":"Coke","quantity":2}]}


In [25]:
import json

arguments = json.loads(message.tool_calls[0].function.arguments)

In [26]:
final_price = calculate_total_price(**arguments)

In [27]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": f"You are a helpful assistant. You tell the client 'Thanks for your ordner. Please drive to the issuing counter and put the money in the machine. The final price is: {final_price}€"},
  ],
)
message = completion.choices[0].message
message

ChatCompletionMessage(content='Thanks for your order. Please drive to the issuing counter and put the money in the machine. The final price is: 15.45€.', role='assistant', function_call=None, tool_calls=None)

In [28]:
from openai import OpenAI
import os

client = OpenAI()

speech_file_path = os.path.join(os.getcwd(), "order.mp3")
print(speech_file_path)
response = client.audio.speech.create(
  model="tts-1",
  voice="alloy",
input=message.content
)
response.stream_to_file(speech_file_path)

c:\Users\User\Desktop\VoiceAssistant\order.mp3


C:\Users\User\AppData\Local\Temp\ipykernel_11536\2404179379.py:13: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)
